In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import math
import h5py
from torch.utils.data import Dataset, DataLoader

In [9]:
if torch.cuda.is_available():
    print(f"CUDA is available. Device: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA is not available.")

CUDA is available. Device: NVIDIA GeForce RTX 4060


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [11]:
class QuantumTransformer(nn.Module):
    def __init__(self, d_model, num_heads, num_layers, dim_feedforward, dropout, seq_length, n_grid, device):
        super(QuantumTransformer, self).__init__()
        
        self.embedding = nn.Linear(3, d_model)  # (Re(Ψ), Im(Ψ), V) → d_model

        # Positional Encoding
        self.positional_encoding = self.create_positional_encoding(seq_length * n_grid, d_model).to(device)

        # Transformer Encoder
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=num_heads, dim_feedforward=dim_feedforward, dropout=dropout
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        # Output layer (maps d_model back to (Re(Ψ), Im(Ψ)))
        self.output_layer = nn.Linear(d_model, 2)

    def create_positional_encoding(self, full_seq_length, d_model):
        """
        Generate a positional encoding tensor of shape (1, full_seq_length, d_model).
        """
        pos_encoding = torch.zeros(full_seq_length, d_model)
        position = torch.arange(0, full_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)
        
        return pos_encoding.unsqueeze(0)  # Shape: (1, full_seq_length, d_model)

    def forward(self, psi_real, psi_imag, potential):
        # Stack input features along last dimension
        x = torch.stack((psi_real, psi_imag, potential), dim=-1)  # Shape: (batch, seq_length, n_grid, 3)
        
        # Flatten the n_grid dimension into the sequence dimension
        batch_size, seq_length, n_grid, _ = x.shape
        x = x.view(batch_size, seq_length * n_grid, 3)

        # Apply embedding
        x = self.embedding(x)  # Shape: (batch, seq_length * n_grid, d_model)
        
        # Add positional encoding (matching seq_length)
        x = x + self.positional_encoding[:, :x.shape[1], :].to(x.device)

        # Transformer Encoder
        x = self.transformer_encoder(x)

        # Apply output layer
        x = self.output_layer(x)  # Shape: (batch, seq_length * n_grid, 2)

        # Reshape back to (batch, seq_length, n_grid, 2)
        x = x.view(batch_size, seq_length, n_grid, 2)
        
        return x


In [12]:
class QuantumWaveDataset(Dataset):
    def __init__(self, h5_file):
        """
        Custom PyTorch Dataset for quantum wavefunction propagation.
        - h5_file: Path to the .h5 file containing dataset_X and dataset_y.
        """
        with h5py.File(h5_file, "r") as f:
            self.X = torch.tensor(f["dataset_X"][:], dtype=torch.float32)  # Shape (num_trajectories, sequence_length, n_grid * 3)
            self.y = torch.tensor(f["dataset_y"][:], dtype=torch.float32)  # Shape (num_trajectories, sequence_length, n_grid * 2)

        # Extract num_trajectories, sequence_length, and n_grid from the shape
        self.num_trajectories, self.sequence_length, n_grid_3 = self.X.shape
        self.n_grid = n_grid_3 // 3  # Since last dimension is n_grid * 3

        # Reshape to (num_trajectories, sequence_length, n_grid, features)
        self.X = self.X.view(self.num_trajectories, self.sequence_length, self.n_grid, 3)
        self.y = self.y.view(self.num_trajectories, self.sequence_length, self.n_grid, 2)

    def __len__(self):
        return self.num_trajectories

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]  # Each sample is (sequence_length, n_grid, features)

# Load dataset
h5_path = "./DataNew/ngrid64_19932025.h5"  # Change this to your actual path
dataset = QuantumWaveDataset(h5_path)

# Create DataLoader for training
batch_size = 10
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = QuantumTransformer(seq_length=dataset.sequence_length, d_model=128, num_heads=8, num_layers=6, dim_feedforward=512, n_grid=64, dropout=0.1, device=device).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

C:\Users\roman\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\nn\modules\transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        # Split inputs into components
        psi_real = X[..., 0]  # (batch_size, seq_length, n_grid)
        psi_imag = X[..., 1]  # (batch_size, seq_length, n_grid)
        potential = X[..., 2] # (batch_size, seq_length, n_grid)

        optimizer.zero_grad()
        output = model(psi_real, psi_imag, potential)  # Output shape: (batch_size, seq_length, n_grid, 2)

        loss = criterion(output, y)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader):.6f}")


Epoch 1/50, Loss: 1.191330
Epoch 2/50, Loss: 0.665365
Epoch 3/50, Loss: 0.379064
Epoch 4/50, Loss: 0.293006
Epoch 5/50, Loss: 0.312087
Epoch 6/50, Loss: 0.341239
Epoch 7/50, Loss: 0.336964
Epoch 8/50, Loss: 0.304093
Epoch 9/50, Loss: 0.257478
Epoch 10/50, Loss: 0.210535
Epoch 11/50, Loss: 0.175002
Epoch 12/50, Loss: 0.154825
Epoch 13/50, Loss: 0.150757
Epoch 14/50, Loss: 0.156608
Epoch 15/50, Loss: 0.166590
Epoch 16/50, Loss: 0.174978
Epoch 17/50, Loss: 0.177761
Epoch 18/50, Loss: 0.173901
Epoch 19/50, Loss: 0.164949
Epoch 20/50, Loss: 0.153228
Epoch 21/50, Loss: 0.142327
Epoch 22/50, Loss: 0.134500
Epoch 23/50, Loss: 0.130680
Epoch 24/50, Loss: 0.131102
Epoch 25/50, Loss: 0.133666
Epoch 26/50, Loss: 0.137510
Epoch 27/50, Loss: 0.140553
Epoch 28/50, Loss: 0.140740
Epoch 29/50, Loss: 0.139102
Epoch 30/50, Loss: 0.135306
Epoch 31/50, Loss: 0.131263
Epoch 32/50, Loss: 0.128352
Epoch 33/50, Loss: 0.125893
Epoch 34/50, Loss: 0.125120
Epoch 35/50, Loss: 0.125563
Epoch 36/50, Loss: 0.126328
E

In [ ]:
import torch

def propagate_wavefunction(model, initial_psi_real, initial_psi_imag, potential, N, device):
    """
    Propagates the wavefunction using the trained Transformer model for N time steps.

    Parameters:
    - model: Trained Transformer model.
    - initial_psi_real: Initial real part of the wavefunction, shape (1, n_grid).
    - initial_psi_imag: Initial imaginary part of the wavefunction, shape (1, n_grid).
    - potential: Potential energy function, shape (N, n_grid).
    - N: Number of time steps to propagate.
    - device: 'cuda' or 'cpu'.

    Returns:
    - propagated_wavefunction: List of predicted wavefunctions [(Re, Im)] for each time step.
    """

    model.eval()  # Set model to evaluation mode
    propagated_wavefunction = []
    
    # Move to device
    psi_real = initial_psi_real.to(device)
    psi_imag = initial_psi_imag.to(device)
    potential = potential.to(device)

    for t in range(N):
        # Extract potential at current time step
        V_t = potential[t].unsqueeze(0)  # Shape: (1, n_grid)
        # Run through the Transformer model
        with torch.no_grad():
            psi_real, psi_imag = model(psi_real, psi_imag, V_t)

        propagated_wavefunction.append((psi_real.cpu().numpy(), psi_imag.cpu().numpy()))

    return propagated_wavefunction

In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def animate_propagation(propagated_wave, dvr_wave, n_grid):
    """
    Animates the wavefunction propagation from the Transformer model compared to DVR.

    Parameters:
    - propagated_wave: List of (Re, Im) wavefunction tuples from the Transformer.
    - dvr_wave: Ground truth wavefunction from the dataset [(Re, Im)].
    - n_grid: Number of spatial grid points.
    """

    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    x = np.linspace(0, 1, n_grid)  # Adjust the spatial domain accordingly
    line1, = ax[0].plot([], [], 'b-', label="Transformer Re(Ψ)")
    line2, = ax[0].plot([], [], 'r-', label="DVR Re(Ψ)")
    ax[0].set_ylim(-1, 1)
    ax[0].set_xlim(x[0], x[-1])
    ax[0].set_title("Real Part of Wavefunction")
    ax[0].legend()

    line3, = ax[1].plot([], [], 'b-', label="Transformer Im(Ψ)")
    line4, = ax[1].plot([], [], 'r-', label="DVR Im(Ψ)")
    ax[1].set_ylim(-1, 1)
    ax[1].set_xlim(x[0], x[-1])
    ax[1].set_title("Imaginary Part of Wavefunction")
    ax[1].legend()

    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        line3.set_data([], [])
        line4.set_data([], [])
        return line1, line2, line3, line4

    def update(frame):
        psi_real_trans, psi_imag_trans = propagated_wave[frame]
        psi_real_dvr, psi_imag_dvr = dvr_wave[frame]

        line1.set_data(x, psi_real_trans)
        line2.set_data(x, psi_real_dvr)
        line3.set_data(x, psi_imag_trans)
        line4.set_data(x, psi_imag_dvr)
        return line1, line2, line3, line4

    ani = animation.FuncAnimation(fig, update, frames=len(propagated_wave), init_func=init, blit=True)
    plt.show()


In [18]:
# Load dataset from .h5 file
dataset_path = h5_path  # Replace with your actual file path

with h5py.File(dataset_path, "r") as f:
    dataset_X = torch.tensor(f["dataset_X"][:], dtype=torch.float32)
    dataset_Y = torch.tensor(f["dataset_y"][:], dtype=torch.float32)

# Move dataset to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dataset_X, dataset_Y = dataset_X.to(device), dataset_Y.to(device)

# Get dataset dimensions
num_trajectories, sequence_length, total_features = dataset_X.shape
n_grid = total_features // 3  # Since input has (Re(Ψ), Im(Ψ), V)


In [19]:
# Select a trajectory (e.g., the first one)
trajectory_idx = 0

# Extract initial real and imaginary parts of wavefunction (t=0)
initial_psi_real = dataset_X[trajectory_idx, 0, :n_grid].unsqueeze(0)  # Shape: (1, n_grid)
initial_psi_imag = dataset_X[trajectory_idx, 0, n_grid:2*n_grid].unsqueeze(0)

# Extract the potential for all timesteps
potential = dataset_X[trajectory_idx, :, 2*n_grid:]  # Shape: (sequence_length, n_grid)

# Extract DVR ground truth wavefunction for comparison
dvr_wave = [(dataset_Y[trajectory_idx, i, :n_grid], dataset_Y[trajectory_idx, i, n_grid:]) for i in range(sequence_length)]

In [24]:
N = sequence_length  # Number of time steps to propagate

# Ensure the tensors have correct shape: (batch_size, seq_length, n_grid)
initial_psi_real = initial_psi_real.unsqueeze(1)  # Shape: (1, 1, n_grid)
initial_psi_imag = initial_psi_imag.unsqueeze(1)  # Shape: (1, 1, n_grid)
potential = potential.unsqueeze(0)  # Shape: (1, sequence_length, n_grid)

# Call the function again
propagated_wave = propagate_wavefunction(model, initial_psi_real, initial_psi_imag, potential, N, device)

RuntimeError: stack expects each tensor to be equal size, but got [1, 1, 1, 64] at entry 0 and [1, 1, 100, 64] at entry 2

In [23]:
print("Initial psi_real shape:", initial_psi_real.shape)  # Should be (1, 1, n_grid)
print("Initial psi_imag shape:", initial_psi_imag.shape)  # Should be (1, 1, n_grid)
print("Potential shape:", potential.shape)  # Should be (sequence_length, n_grid)

Initial psi_real shape: torch.Size([1, 1, 64])
Initial psi_imag shape: torch.Size([1, 1, 64])
Potential shape: torch.Size([1, 100, 64])


In [20]:
N = sequence_length  # Number of time steps to propagate

# Ensure tensors are on the correct device
initial_psi_real, initial_psi_imag, potential = initial_psi_real.to(device), initial_psi_imag.to(device), potential.to(device)

# Propagate wavefunction
propagated_wave = propagate_wavefunction(model, initial_psi_real, initial_psi_imag, potential, N, device)

# Animate the propagation
animate_propagation(propagated_wave, dvr_wave, n_grid)

ValueError: not enough values to unpack (expected 4, got 3)

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=1000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # Shape: (1, max_len, d_model)

    def forward(self, x):
        return x + self.pe[:, :x.shape[1], :].to(x.device)

class QuantumTransformer(nn.Module):
    def __init__(self, n_grid, d_model=128, num_layers=4, num_heads=8, dim_feedforward=256, dropout=0.1):
        super(QuantumTransformer, self).__init__()
        self.n_grid = n_grid
        self.d_model = d_model
        
        # Linear projection of inputs
        self.input_proj = nn.Linear(3, d_model)  # (psi_real, psi_imag, V) -> d_model
        self.output_proj = nn.Linear(d_model, 2) # d_model -> (psi_real, psi_imag)
        
        # Positional encoding
        self.positional_encoding = PositionalEncoding(d_model)
        
        # Transformer encoder
        encoder_layer = nn.TransformerEncoderLayer(d_model, num_heads, dim_feedforward, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)
        
    def forward(self, psi_real, psi_imag, potential, mask=None):
        batch_size, seq_length, n_grid = psi_real.shape
        
        # Stack inputs along last dimension: (batch, seq_length, n_grid, 3)
        x = torch.stack((psi_real, psi_imag, potential), dim=-1)
        x = x.view(batch_size, seq_length * n_grid, 3)  # Flatten spatial dim
        
        # Apply input projection and positional encoding
        x = self.input_proj(x)
        x = self.positional_encoding(x)
        
        # Transformer encoder forward pass
        x = self.transformer_encoder(x, mask)
        
        # Reshape and apply output projection
        x = self.output_proj(x)  # Shape: (batch, seq_length * n_grid, 2)
        x = x.view(batch_size, seq_length, n_grid, 2)  # Reshape back
        
        psi_real_pred, psi_imag_pred = x[..., 0], x[..., 1]
        return psi_real_pred, psi_imag_pred

# Move model to CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
n_grid = 100  # Example grid size
model = QuantumTransformer(n_grid).to(device)


cuda
